<a href="https://colab.research.google.com/github/CarlosLeandro09/DataAnalysisRadiology/blob/main/Osprey_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Please click on the FOLDER icon next to it and upload the files to the session storage. Follow the instructions:


In [ ]:
pip install python-magic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m pip install dcm2niix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dcm2niix-1.0.20220715.tar.gz (451 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dcm2niix: filename=dcm2niix-1.0.20220715-cp310-cp310-linux_x86_64.whl size=576078 sha256=887ece96adc9fc5bc48f86e6eb6fb6ce8f6a28fa535850ed5267e4f0da52f4f8
  Stored in directory: /root/.cache/pip/wheels/88/8d/9b/5ea20c0451a1acddef585757be7dfec121ee076e58503b267c
Successfully built dcm2niix


In [ ]:
import zipfile
import os
import numpy as np
import magic
import shutil
import subprocess
import shutil, errno

print("Hello, welcome to the file organization algorithm for processing in Osprey!")
workdir = '/content/'
sub = int(input('Please enter the number of subjects: '))
t1w = np.zeros(sub, dtype = object)
spec = np.zeros(sub, dtype = object)
subs = ["%.2d" % i for i in range(1,sub+1)]
for i in subs:
  path = os.path.join(workdir, 'sub-' + i)
  os.mkdir(path)

se = int(input('Please enter the number of sessions: '))
tp = input('Please enter the name of the pulse sequence used in the acquisition: ')
ses = ["%.2d" % i for i in range(1,se+1)]
for a in subs:
  for b in ses:
    path = os.path.join(workdir + 'sub-' + a, 'ses-' + b)
    os.mkdir(path)
    os.mkdir(path + '/mrs')
    os.mkdir(path + '/anat')
    os.mkdir(path + '/anat/' + 'sub-' + a)
    os.mkdir(path + '/mrs/' + '/'+ 'sub-' + a+ '-ses-' + b +'_' + tp + '-ref')
    os.mkdir(path + '/mrs/' + '/'+ 'sub-' + a+ '-ses-' + b +'_' + tp)
zip_ = input("Are your files in .zip format? [y or n] ")
print("...")

names = []

if zip_ == 'y':
  for j in os.listdir(workdir):
    if j.endswith(".zip"):
      with zipfile.ZipFile(workdir + j, 'r') as zip_ref:
        zip_ref.extractall(workdir)
        u = j.replace(" ", "")
        os.rename((workdir + j)[:-4], (workdir + u)[:-4])
        names.append(u)
else:
  pass

for k in range(1,sub+1):
  for l in range(1,se+1):
    t1w[k-1] = input('Please provide the directory that contains the T1-weighted images for subject number ' + str(k) + ' and for section number ' + str(l) + ': ')
    command = os.popen('dcm2niix -f %n_%p_%s -z i -m y -s y -ba y -b y -p n '+ str(t1w[k-1]))
    print(command.read())
    print(command.close())
    for c in os.listdir(t1w[k-1]):
      final_path = t1w[k-1] + '/' + c
      if c.endswith(".nii.gz"):
        shutil.copy(final_path, workdir + 'sub-' + subs[k-1] + '/ses-' + ses[l-1] + '/anat/sub-'+ subs[k-1])
        for t1w in os.listdir(workdir + 'sub-' + subs[k-1] + '/ses-' + ses[l-1] + '/anat/sub-'+ subs[k-1]):
          if t1w.endswith(".nii.gz"):
            os.rename(workdir + 'sub-' + subs[k-1] + '/ses-' + ses[l-1] + '/anat/sub-'+ subs[k-1] + '/' + t1w, workdir + 'sub-' + subs[k-1] + '/ses-' + ses[l-1] + '/anat/sub-'+ subs[k-1] + '/' +  'sub-' + subs[k-1] + '_ses-' + ses[l-1] + '_T1w.nii.gz')
print("...")
print("Take your time, we're completing the process...")
for m in range(1,sub+1):
  for n in range(1,se+1):
    spec[m-1] = input('Please provide the path to the file containing spectroscopic data, excluding reference data, for the subject identified by the corresponding number ' + str(m) + ': ')
    shutil.copy(spec[m-1], workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp)
    shutil.copy(spec[m-1][:-5]+'.SPAR', workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp)
    for mrs in os.listdir(workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp):
      if mrs.endswith(".SDAT"):
        os.rename(workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '/' + str(mrs), workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '/' +  'sub-' + subs[k-1] + '_'+ tp + '_act.SDAT')
        os.rename(workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '/' + str(mrs[:-5] + '.SPAR'), workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '/' +  'sub-' + subs[k-1] + '_'+ tp + '_act.SPAR')
for m in range(1,sub+1):
  for n in range(1,se+1):
    spec[m-1] = input('Please provide the path to the reference file containing spectroscopic data for the subject identified by the corresponding number ' + str(m) + ': ')
    shutil.copy(spec[m-1], workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '-ref')
    shutil.copy(spec[m-1][:-5]+'.SPAR', workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '-ref')
    for mrs in os.listdir(workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp+ '-ref'):
      if mrs.endswith(".SDAT"):
        os.rename(workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '-ref/' + str(mrs), workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp +  '-ref/' +  'sub-' + subs[k-1] + '_'+ tp + '_ref.SDAT')
        os.rename(workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '-ref/' + str(mrs[:-5] + '.SPAR'), workdir + 'sub-' + subs[m-1] + '/ses-' + ses[n-1] + '/mrs/'+ 'sub-' + subs[m-1] + '-ses-' + ses[n-1] + '_' +tp + '-ref/' +  'sub-' + subs[k-1] + '_'+ tp + '_ref.SPAR')
def copyanything(src, dst):
    try:
        shutil.copytree(src, dst)
    except OSError as exc:
        if exc.errno in (errno.ENOTDIR, errno.EINVAL):
            shutil.copy(src, dst)
        else: raise

os.mkdir(workdir + 'Final')
for b in range(1,sub+1):
  copyanything(workdir + 'sub-' + subs[b-1], workdir + 'Final/'+ 'sub-' + subs[b-1])
  archived = shutil.make_archive('Data', 'zip', '/content/Final')

print('Done! Please download the .zip file.')

Hello, welcome to the file organization algorithm for processing in Osprey!
Please enter the number of subjects: 1
Please enter the number of sessions: 1
Please enter the name of the pulse sequence used in the acquisition: PRESS
Are your files in .zip format? [y or n] y
...
Please provide the directory that contains the T1-weighted images for subject number 1 and for section number 1: /content/DelRec-3DT1/DICOM
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 183 DICOM file(s)
Skipping non-image DICOM: /content/DelRec-3DT1/DICOM/XX_0183
Skipping non-image DICOM: /content/DelRec-3DT1/DICOM/XX_0001
Philips Scaling Values RS:RI:SS = 1.93016:0:0.000214879 (see PMC3998685)
Convert 180 DICOM as /content/DelRec-3DT1/DICOM/RAC_WIP_DelRec_-_3DT1_iso1mm_203 (240x240x180x1)
Conversion required 2.470194 seconds (2.456251 for core code).

None
...
Take your time, we're completing the process...
Please provide the path to the file containing spectroscopic data, exc